# Packages
---

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.inspection import permutation_importance
import xgboost as xgb
import joblib
import plotly.express as px
from scipy.stats import binom
import warnings
warnings.filterwarnings('ignore')
import plotly
plotly.offline.init_notebook_mode(connected=True)

seed = 123

# Load data
---
In addition to reading the dataset, we will also filter the columns, keeping those that will be part of our predictive space.
We will also store the xG provided by Statsbomb for future comparison.

> **We will save the matches of the Spanish team in the World Cup to later simulate what the result of their respective matches would have been according to the predicted xG value of the shots in each of the matches.**

In this way, we will consider two sets of data: `train_df` and `eval_df`, being eval_df the releases that took place in the matches in which Spain participated in the World Cup.

> We will add a new column that indicates if the throw has been from a FCBarcelona player because later we will be interested in balancing Barcelona's shots with the rest of the teams.

In [12]:
df = pd.read_csv('../output_files/LaLiga_WorldCup/final_df.csv', sep=";")

df['isPlayerAssisted'] = df['playerAssisted'].apply(lambda x: 1 if x != 'None' else 0)
df['isBarcelonaShot'] = df['playerTeam'].apply(lambda x: 'isBarcelona' if x == 'Barcelona' else 'notBarcelona')

print("Number of Barcelona shots:", df[df.isBarcelonaShot == 'isBarcelona'].shape[0])
print("Number of Barcelona's rivals shots:", df[df.isBarcelonaShot == 'notBarcelona'].shape[0])

#df_eval = df[(df.playerTeam == 'Spain') | (df.opponentTeam == 'Spain')].reset_index(drop=True)
df_eval = df[df.competitionSeason == 'FIFA World Cup (2018)']

attributes = ['distance','angle','playType','situation','playerPosition','bodyPart','technique','underPressure',
              'firstTimeCarry','isPlayerAssisted','keyPassType', 'previousKeyPass', 'playersBetweenShotAndGoal',
              'xGoalkeeper','yGoalkeeper','isBarcelonaShot','xG','target']

eval_df = df_eval[attributes]
eval_df = eval_df[[c for c in eval_df if c != 'isBarcelonaShot']]
#train_df = df[(df.playerTeam != 'Spain') & (df.opponentTeam != 'Spain')].reset_index(drop=True)[attributes]
train_df = df[df.competitionSeason != 'FIFA World Cup (2018)'].reset_index(drop=True)[attributes]

Number of Barcelona shots: 7639
Number of Barcelona's rivals shots: 6037


In [13]:
train_df.shape

(11983, 18)

In [14]:
eval_df.shape

(1693, 17)

## Barcelona Proportions

In [15]:
fig = go.Figure(go.Bar(x = ["Barcelona", "Barcelona's rivals"], 
                       y = [train_df[train_df.isBarcelonaShot == 'isBarcelona'].shape[0] / train_df.shape[0], 
                            train_df[train_df.isBarcelonaShot == 'notBarcelona'].shape[0] / train_df.shape[0]], 
                       marker_color = 'chocolate'))
fig.update_layout(title_text = 'Team shots proportions', xaxis_title = 'Team', yaxis_title = 'percentage')

## Target Class Proportions

In [16]:
fig = go.Figure(go.Bar(x = pd.DataFrame(train_df['target'].value_counts(normalize=True)).sort_index().index, 
                       y = pd.DataFrame(train_df['target'].value_counts(normalize=True)).sort_index()['target'], 
                      marker_color = 'chocolate'))
fig.update_layout(title_text = 'Target Class Proportions', xaxis_title = 'target', yaxis_title = 'percentage')

In [17]:
train_df.target.value_counts()

notGoal    10344
isGoal      1639
Name: target, dtype: int64

# Imbalanced Classification
---

## Up-Sample Minority Class

Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal.

Next, we'll create a new DataFrame with an up-sampled minority class. Here are the steps:

1. First, we'll separate observations from each class into different DataFrames.
2. Next, we'll resample the minority class with replacement, setting the number of samples to match that of the majority class.
3. Finally, we'll combine the up-sampled minority class DataFrame with the original majority class DataFrame.

In [18]:
df_majority = train_df[train_df.target=='notGoal']
df_minority = train_df[train_df.target=='isGoal']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,                      # sample with replacement
                                 n_samples=df_majority.shape[0],    # to match majority class
                                 random_state=seed)                 # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.target.value_counts()

isGoal     10344
notGoal    10344
Name: target, dtype: int64

> This may balance the class distribution, but does **not provide any additional information to the model.**

## SMOTE

An improvement in the duplication of minority class examples is to **synthesize new minority class examples.**

Perhaps the most widely used approach to synthesizing new examples is called the synthetic minority oversampling technique, or `SMOTE` for short. 

**SMOTE** works by selecting nearby examples in the feature space, drawing a line between the examples in the feature space, and drawing a new sample at a point along that line.

Specifically, a random example of the minority class is chosen first. Then k of the nearest neighbors are found for that example (typically k = 5 ). A randomly selected neighbor is chosen and a synthetic example is created at a randomly selected point between the two examples in the feature space.

> In our example we will seek to balance the class isBarcelonaShot so that we have a similar proportion between shots from barcelona and the rest of the teams.

# Data Preparation and Data Preprocess
---

Within this stage we can carry out the following phases:

1. Separation of training and eval data.
2. Conversion of categorical variables to numerical variables.
3. Standardization of numerical variables (StandardScaler)

## Separation of training and eval data

At this stage, we remove the `xG` attribute from our predictive space.

In [19]:
X_train = train_df[[c for c in train_df.columns[:-3]]]
xG_train = train_df['xG']
barcelonaShot_train = train_df['isBarcelonaShot']
y_train = train_df['target']

X_eval = eval_df[[c for c in eval_df.columns[:-2]]]
xG_eval = eval_df['xG']
y_eval = eval_df['target']

print("Target class proportions in each dataset:")
print("Training set:", y_train.value_counts(normalize=True).to_dict())
print("Eval data to simulate match:", y_eval.value_counts(normalize=True).to_dict())

Target class proportions in each dataset:
Training set: {'notGoal': 0.8632228991070684, 'isGoal': 0.13677710089293166}
Eval data to simulate match: {'notGoal': 0.8930891907855877, 'isGoal': 0.10691080921441229}


In [20]:
features_cat = [c for c in X_train.columns if X_train.dtypes[c] == object]
features_num = list(X_train.columns[X_train.dtypes != object])

features_columns = features_num + features_cat

X_train = X_train[features_columns]
X_eval = X_eval[features_columns]

print("Number of Features:")
print("- Categorical:", len(features_cat))
print("- Numerical:", len(features_num))

Number of Features:
- Categorical: 8
- Numerical: 7


## Data Preprocessing

### Extraction of the numerical and categorical variables

In [21]:
class DataFrameSelector(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names: list):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names]

### Pipeline definition for numerical and categorical variables

We are now in a position to define the 'pipeline' corresponding to each subset of variables where we will indicate the transformation carried out both for the set of numerical variables and for the categorical set.

#### Transformers and `CategoricalEncoder()`

We have decided to use the `StandardScaler()` function as a numerical transformer and 'CategoricalEncoder()` as a categorical transformer which we will see below its implementation.


In [22]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        self.frecuency = pd.Series([X[c].value_counts() for c in X], 
                                    index=X.columns)
        list_clases = [self.frecuency[i].index.to_list() for i in range(len(self.frecuency))]
        clases = [self.frecuency[i].index.to_list() for i in range(len(self.frecuency))]
        self.clases = [item for sublist in clases for item in sublist]
        return self
    
    def transform(self, X):
        freq_new = pd.Series([X[c].value_counts() for c in X], index = X.columns)
        for c in X:
            dic = {}
            i=1
            for label in self.frecuency[c].index:
                dic[label] = i
                i += 1
            for label in freq_new[c].index:
                if label not in self.clases:
                    dic[label] = i
            X[c] = X[c].map(dic)
        return X

1. We will calculate the frequency of each of the categories of our categorical variables.
2. Next, we will order the categories according to this frequency per variable and assign the value 1 to the most frequent category, the value 2 to the second most frequent one, etc.
2. Finally, we will convert our categorical variables into numerical variables according to this rule, that is: a variable with 20 categories will be between 1 and 19 and the observations that take the value 1 will be because they presented the most frequent category in those variables.

**What do we get with this?** Firstly, we avoid having to create new variables that hinder the computational capacity of our models. Secondly, taking into account the tree-based models, by considering a variable of this type as a division of the tree, we will be distinguishing between more frequent and less frequent categories, so our tree will be able to choose between several values, being able to grow in both directions, eliminating the problem described with the previous coding.

#### Combination of the pipelines

But how do we combine the two pipelines?

Well, to do that we'll use the scikit-learn `FeatureUnion` function. This way, giving this class as an argument the two built pipelines with their corresponding variables to transform, we can call the `fit_transform` method so that the transformations are done separately and then combine and return the results.

This way of carrying out the preprocessing of the data is much more efficient than any script that deals with this type of preprocessing as we are defining a structure that will take care of doing this preprocessing as many times as we want. Our task will just be to call the `fit_transform' method and enter the set of data we want to preprocess.

* **Pipeline creation**

In [23]:
numeric_features = [c for c in X_train.columns if X_train[c].dtypes != 'object']
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

categorical_pipeline = Pipeline(steps = [('cat_selector', DataFrameSelector(categorical_features)), 
                                         ('one_hot_encoder', CategoricalEncoder())])
categorical_pipeline_ohe = Pipeline(steps = [('cat_selector', DataFrameSelector(categorical_features)), 
                                             ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', 
                                                                               sparse=False))])

numerical_pipeline = Pipeline(steps = [('num_selector', DataFrameSelector(numeric_features)),
                                       ('std_scaler', StandardScaler())])

preprocess_pipeline = FeatureUnion(transformer_list = [('categorical_pipeline', categorical_pipeline),
                                                         ('numerical_pipeline', numerical_pipeline)])

preprocess_pipeline_ohe = FeatureUnion(transformer_list = [('categorical_pipeline', categorical_pipeline_ohe),
                                                           ('numerical_pipeline', numerical_pipeline)])

> **Transformation of data sets**

In [24]:
preprocessing_pl = preprocess_pipeline.fit(X_train)
preprocessing_pl_ohe = preprocess_pipeline_ohe.fit(X_train)

> **Ordinal Encoder**

In [25]:
train_X = preprocessing_pl.transform(X_train)

eval_X = preprocessing_pl.transform(X_eval)

train_df = pd.concat([pd.DataFrame(train_X, columns = categorical_features + numeric_features), 
                      pd.concat([barcelonaShot_train, y_train], axis=1)], axis=1)
eval_df = pd.concat([pd.DataFrame(eval_X, columns =  categorical_features + numeric_features), 
                     pd.DataFrame(y_eval, columns=['target']).reset_index(drop=True)], axis=1)

> **OneHotEncoder**

In [26]:
train_ohe_X = preprocessing_pl_ohe.transform(X_train)

In [27]:
dict_reps = {name: X_train[name].nunique() for name in categorical_features}

name_categorical_ohe = []
for key in dict_reps.keys():
    name_categorical_ohe.extend(list(np.repeat(key, dict_reps[key])))
    
attributes_names_ohe = [name + type_name[2:] for name, type_name 
                        in zip(name_categorical_ohe, list(categorical_pipeline_ohe.\
                                                          named_steps['one_hot_encoder'].get_feature_names()))]

In [28]:
train_ohe_df = pd.concat([pd.DataFrame(train_ohe_X, columns = attributes_names_ohe + numeric_features), 
                          pd.concat([barcelonaShot_train, y_train], axis=1)], axis=1)

In [29]:
eval_ohe_df = pd.concat([pd.DataFrame(preprocessing_pl_ohe.transform(X_eval), columns=attributes_names_ohe + numeric_features), 
                         pd.DataFrame(y_eval, columns=['target']).reset_index(drop=True)], axis=1)

* **Drop last categorical dummy variable for each categorical variable original (multicoliniality)**

In [30]:
index_cols_to_drop = [i-1 for i in list(np.cumsum(list(dict_reps.values())))]
columns_to_drop = train_ohe_df.columns[index_cols_to_drop]

In [31]:
columns_to_drop

Index(['playType_Penalty', 'situation_Regular Play', 'playerPosition_Striker',
       'bodyPart_Right Foot', 'technique_Volley', 'firstTimeCarry_firstTime',
       'keyPassType_None', 'previousKeyPass_forward'],
      dtype='object')

In [32]:
train_ohe_df = train_ohe_df.drop(columns_to_drop, axis=1)

In [33]:
eval_ohe_df = eval_ohe_df.drop(columns_to_drop, axis=1)

In [34]:
eval_ohe_df.to_csv('../output_files/FIFA_World_Cup_preprocess_ohe.csv', sep=";", index=False)

In [35]:
train_ohe_df.head()

,playType_Corner,playType_Free Kick,playType_Open Play,situation_From Corner,situation_From Counter,situation_From Free Kick,situation_From Goal Kick,situation_From Keeper,situation_From Kick Off,situation_From Throw In,...,previousKeyPass_cut-back,distance,angle,underPressure,isPlayerAssisted,playersBetweenShotAndGoal,xGoalkeeper,yGoalkeeper,isBarcelonaShot,target
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.391486,-0.788139,-0.443289,0.630682,-0.620273,0.168557,1.038720,isBarcelona,notGoal
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.459238,-0.735564,-0.443289,-1.585584,2.290245,0.642288,-0.032237,isBarcelona,notGoal
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,-1.055146,1.236016,2.255864,0.630682,-0.620273,0.082424,0.110557,isBarcelona,isGoal
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.298501,-0.847287,-0.443289,-1.585584,1.562615,0.556155,0.074859,isBarcelona,notGoal
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,-0.595898,0.473672,-0.443289,0.630682,-0.620273,-0.520507,0.074859,isBarcelona,notGoal


In [36]:
eval_df.to_csv('../output_files/FIFA_World_Cup_preprocess.csv', sep=";", index=False)

## Division of the training set

We generate a partition of our training set consisting of 30% of the total size to evaluate the goodness of our model.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(train_ohe_df.iloc[:,:-1], train_ohe_df.target, test_size = 0.3, 
                                                    stratify=train_ohe_df.target, random_state = seed)

print("Target class proportions in each dataset:")
print("- Training set:", y_train.value_counts(normalize=True).to_dict())
print("- Testing set:", y_test.value_counts(normalize=True).to_dict())

print("Number of records:")
print("- Training set:", X_train.shape[0])
print("- Test set:", X_test.shape[0])

Target class proportions in each dataset:
- Training set: {'notGoal': 0.8632570338578922, 'isGoal': 0.13674296614210776}
- Testing set: {'notGoal': 0.8631432545201669, 'isGoal': 0.1368567454798331}
Number of records:
- Training set: 8388
- Test set: 3595


## Imbalanced Technique: `SMOTE`

The original paper on `SMOTE` suggested **combining `SMOTE` with random subsampling of the majority class.**

The unbalanced learning library supports random subsampling through the `RandomUnderSampler class`.

We will first subsample the minority class to get 20 percent of the number of examples in the majority class, then use random subsampling to reduce the number of examples in the majority class to get 60 percent more than the minority class.

_**This technique of balancing the variable to be predicted will only be applied to the training set**_

We will apply the SMOTE technique to balance the Barcelona and non-Barcelona records over the training set (regardless of the target value over these records).
Although we are facing an unbalanced dataset, we are not interested in balancing the sample because we would be giving the model wrong behavior patterns.

In [38]:
full_train = pd.concat([X_train, y_train], axis=1)
full_train['target'] = full_train['target'].apply(lambda x: 1 if x=='isGoal' else 0)
full_train = full_train[[c for c in full_train.columns if c!='isBarcelonaShot']+['isBarcelonaShot']]

over = SMOTE()
steps = [('o', over)]
pipeline = Pipeline(steps=steps)
X_train_Smote_Barcelona, y_train_Smote_Barcelona = pipeline.fit_resample(full_train.iloc[:,:-1], full_train.iloc[:,-1])

In [39]:
fig = go.Figure(go.Bar(x=['Barcelona Shot', 'Not Barcelona Shot'], 
                       y=pd.DataFrame(y_train_Smote_Barcelona.value_counts(normalize=True))['isBarcelonaShot'], 
                       marker_color='indianred'))
fig.update_layout(title_text = 'Proportions teams shots')
fig.show()

* **Data Preparation**

We must now consider the set of characteristics and the target variable to be predicted.

In [40]:
X_train_final = X_train_Smote_Barcelona.iloc[:,:-1]
y_train_final = X_train_Smote_Barcelona.iloc[:,-1]
X_test = X_test.iloc[:,:-1]
y_train_final = y_train_final.apply(lambda x: 'isGoal' if x==1 else 'notGoal')

print("Number of records in the training set after the synthetic records have been created:", X_train_final.shape[0])

Number of records in the training set after the synthetic records have been created: 10602


In [41]:
X_train_final.to_csv('../output_files/LaLiga_WorldCup/files_to_models/X_train.csv', sep=";", index=False)
pd.DataFrame(y_train_final, columns=['target'])\
             .to_csv('../output_files/LaLiga_WorldCup/files_to_models/y_train.csv', sep=";", index=False)
X_test.to_csv('../output_files/LaLiga_WorldCup/files_to_models/X_test.csv', sep=";", index=False)
pd.DataFrame(y_test, columns=['target'])\
             .to_csv('../output_files/LaLiga_WorldCup/files_to_models/y_test.csv', sep=";", index=False)

In [42]:
X_train_final.to_csv('../output_files/LaLiga_WorldCup/files_to_models/X_train_ohe.csv', sep=";", index=False)
pd.DataFrame(y_train_final, columns=['target'])\
             .to_csv('../output_files/LaLiga_WorldCup/files_to_models/y_train.csv', sep=";", index=False)
X_test.to_csv('../output_files/LaLiga_WorldCup/files_to_models/X_test_ohe.csv', sep=";", index=False)
pd.DataFrame(y_test, columns=['target'])\
             .to_csv('../output_files/LaLiga_WorldCup/files_to_models/y_test.csv', sep=";", index=False)

In [43]:
y_test = y_test.apply(
    lambda x: 1 if x == 'isGoal' else 0)
y_train_final = y_train_final.apply(
    lambda x: 1 if x == 'isGoal' else 0)

# Supervised Models
---

## Functions

In [48]:
def hyperparameters_optimization(model, num_folds, param_grid, X_train, y_train, grid):
    kfold = KFold(n_splits=num_folds, random_state = seed)
    if grid:
        grid_model = GridSearchCV(estimator = model, param_grid = param_grid, cv = kfold, scoring = 'roc_auc')
    else:
        grid_model = RandomizedSearchCV(estimator = model, param_distributions = param_grid, 
                                        cv = kfold, scoring = 'roc_auc', n_iter = 10)
    grid_model_fit = grid_model.fit(X_train, y_train)
    print("Best: %f using %s" % (grid_model_fit.best_score_, grid_model_fit.best_params_))
    return grid_model_fit.best_estimator_


def train_model_predict(best_model, X_test):
    y_test_pred = best_model.predict(X_test)
    y_test_proba = best_model.predict_proba(X_test)
    auc_score = round(roc_auc_score(y_test,y_test_proba[:,1]),3)
    return y_test_pred, y_test_proba, auc_score


def plot_roc_curve(y, y_model_proba, model):
    fpr, tpr, thresholds = roc_curve(y, y_model_proba[:,1])#, pos_label='isGoal')
    auc_score = round(roc_auc_score(y, y_model_proba[:,1]),3)
    fig = go.Figure()
    legend_curve = model + ' (AUC = ' + str(auc_score) + ')'
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='indianred')))
    fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', line=dict(color='royalblue'), showlegend=False))
    fig.update_layout(title_text = 'Area Curva ROC TEST', 
                      xaxis_title = 'False Positive Rate', yaxis_title = 'True Positive Rate')
    fig.show()
    return fig


def metrics(y_test, y_test_pred, auc_score, model):
    cm = confusion_matrix(y_test, y_test_pred).tolist()
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
    metrics = {'Precision': round(tp / (tp+fp), 2), 'Recall (TPR)': round(tp / (tp+fn), 2), 
               'FPR': round(fp / (fp+tn), 2), 'AUC': auc_score}
    fig = go.Figure(go.Heatmap(x = ["notGoal", "isGoal"], y = ["notGoal", "isGoal"], z = cm, colorscale="Viridis"))
    fig.update_layout(title_text = 'Confusion Matrix', xaxis_title = 'Predicted Class', yaxis_title = 'Real Class', 
                      annotations = [
                      dict(x = 'notGoal', y = 'notGoal', text = str(cm[0][0]), font = {'color': 'white'}, 
                           xref = 'x1', yref = 'y1', showarrow = False), 
                      dict(x = 'isGoal', y = 'notGoal', text = str(cm[0][1]), font = {'color': 'white'},
                           xref = 'x1', yref = 'y1', showarrow = False),
                      dict(x = 'notGoal', y = 'isGoal', text = str(cm[1][0]), font = {'color': 'white'},
                           xref = 'x1', yref = 'y1', showarrow = False),
                      dict(x = 'isGoal', y = 'isGoal', text = str(cm[1][1]), font = {'color': 'white'},
                           xref = 'x1', yref = 'y1', showarrow = False),
                  ])
    fig.show()
    metrics_dataframe = pd.DataFrame.from_dict(metrics, orient='index', columns = [model])
    return metrics_dataframe

def feature_importances_model(model_name, best_model, X_train_final, y_train_final):
    import pandas as pd
    import os
    import joblib
    import plotly.express as px
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import confusion_matrix
    from sklearn.inspection import permutation_importance
    if (model_name == 'KNeighbors') | (model_name == 'SVC'):
        results = permutation_importance(best_model, X_train_final, 
                                         y_train_final, scoring='roc_auc')
        feature_importances = results.importances_mean.reshape(1,X_train_final.shape[1])
        label = 'importance'
        title = 'Permutation Feature Importance (' + model_name + ')'
    else:
        if model_name == 'Logistic Regression':
            feature_importances = best_model.coef_
            label = 'coef'
            title = model_name + ' Feature Importance'
        else:
            feature_importances = best_model.feature_importances_.reshape(1,X_train_final.shape[1])
            label = 'importance'
            title = 'Decision Tree Feature Importance (' + model_name + ')'
    importance = pd.DataFrame(feature_importances, columns=X_train_final.columns).T
    importance.columns = [label]
    importance.index.name = 'attributes'
    importance = importance.sort_values(by=label).reset_index()
    fig = px.bar(importance, x=label, y='attributes', color = label, 
                 orientation='h')
    fig.update_layout(title_text = title)
    return fig

## Metrics Evaluation

$$TPR (True Positive Rate) = \frac{TP}{TP+FN}$$
$$FPR (False Positive Rate) = \frac{FP}{FP+TN}$$

`Precision:` measures the probability of a sample classified as positive to actually be positive.
$$Precision = \frac{TP}{TP+FP}$$

`Recall = TPR (True Positive Rate)`: measures the probability of a sample real as positive to actually be positive.
$$Recall = \frac{TP}{TP+FN}$$

> _**In our example the positive class is `isGoal` and the negative class `notGoal`**_

> If the number of negative samples is very large (a.k.a imbalance data set) the `false positive rate` increases more slowly. Because the true negatives (in the fpr denominator — (FP+TN)) would probably be very high and make this metric smaller.
`Precision` however, is not affected by a large number of negative samples, that’s because it measures the number of true positives out of the samples predicted as positives (TP+FP).

> `Precision` is more focused in the positive class than in the negative class, it actually measures the probability of correct detection of positive values, while FPR and TPR (ROC metrics) measure the ability to distinguish between the classes.

**Our target variable is clearly unbalanced. However, it is not a good option to balance this variable because we would be telling the model that from 10 shots to goal, 5 would be goal when the probability of goal will always be lower than the probability of failure by nature.**

## Logistic Regression

In [45]:
auc_metrics_models = dict()

best_logit = hyperparameters_optimization(model = LogisticRegression(), num_folds = 3,
                                          param_grid = {'C':[0.0001,0.001,0.01,0.1,1,10,100]}, 
                                          X_train = X_train_final, y_train = y_train_final, grid = True)

y_test_pred_lr, y_test_proba_lr, auc_score_lr = train_model_predict(best_logit, X_test)

auc_curve = plot_roc_curve(y_test, y_test_proba_lr, 'Logistic Regression')

Best: 0.829263 using {'C': 0.1}


In [46]:
metrics_lr = metrics(y_test = y_test, y_test_pred = y_test_pred_lr, 
                              auc_score = auc_score_lr, model = 'Logistic Regression')

In [49]:
feature_importances_model('Logistic Regression', best_logit, X_train_final, y_train_final)

## kNN


In [50]:
best_kNN = hyperparameters_optimization(model = KNeighborsClassifier(), num_folds = 3,
                                        param_grid = dict(n_neighbors=[3,5,7,9,11]), 
                                        X_train = X_train_final, y_train = y_train_final, grid = True)

y_test_pred_knn, y_test_proba_knn, auc_score_knn = train_model_predict(best_kNN, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_knn[:,1])
legend_curve = 'KNeighbors (AUC = ' + str(auc_score_knn) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='darkgreen')))

Best: 0.789829 using {'n_neighbors': 11}


In [51]:
metrics_knn = metrics(y_test = y_test, y_test_pred = y_test_pred_knn, auc_score = auc_score_knn, model = 'KNeighbors')

In [52]:
metrics_models = pd.concat([metrics_lr, metrics_knn], axis = 1)
metrics_models

,Logistic Regression,KNeighbors
Precision,0.700,0.64
Recall (TPR),0.250,0.23
FPR,0.020,0.02
AUC,0.808,0.75


In [53]:
feature_importances_model('KNeighbors', best_kNN, X_train_final, y_train_final)

## SVM

In [54]:
best_svm = hyperparameters_optimization(model = SVC(probability=True), num_folds = 3,
                                        param_grid = {'kernel':['rbf', 'sigmoid'], 
                                                      'C':np.logspace(np.log10(0.001), np.log10(200), num=20), 
                                                      'gamma':np.logspace(np.log10(0.00001), np.log10(2), num=30)}, 
                                        X_train = X_train_final, y_train = y_train_final, grid=False)

y_test_pred_svm, y_test_proba_svm, auc_score_svm = train_model_predict(best_svm, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_svm[:,1])
legend_curve = 'SVC (AUC = ' + str(auc_score_svm) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='orange')))

Best: 0.792051 using {'kernel': 'rbf', 'gamma': 0.00044172017955784455, 'C': 0.0019010851906124285}


In [55]:
metrics_svm = metrics(y_test = y_test, y_test_pred = y_test_pred_svm, auc_score = auc_score_svm, model = 'SVC')
metrics_models = pd.concat([metrics_models, metrics_svm], axis = 1)

In [56]:
feature_importances_model('SVC', best_svm, X_train_final, y_train_final)

## Decision Tree

In [57]:
tree_grid = {"min_samples_split": [2, 5, 10, 20],
            "max_depth": [None, 6, 8, 20],
            "min_samples_leaf": [1, 2, 5, 10],
            "max_leaf_nodes": [None, 5, 20, 100]}

best_tree = hyperparameters_optimization(model = DecisionTreeClassifier(random_state = seed), 
                                         num_folds = 3, param_grid = tree_grid, 
                                         X_train = X_train_final, y_train = y_train_final, grid=True)

y_test_pred_tree, y_test_proba_tree, auc_score_tree = train_model_predict(best_tree, X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_tree[:,1])
legend_curve = 'Decision Tree (AUC = ' + str(auc_score_tree) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines'))

Best: 0.802651 using {'max_depth': 6, 'max_leaf_nodes': None, 'min_samples_leaf': 5, 'min_samples_split': 20}


In [58]:
metrics_tree = metrics(y_test = y_test, y_test_pred = y_test_pred_tree, auc_score = auc_score_tree, model = 'Decision Tree')
metrics_models = pd.concat([metrics_models, metrics_tree], axis = 1)

In [59]:
feature_importances_model('Decision Tree', best_tree, X_train_final, y_train_final)

## Ensemble Methods

### Extra Trees

In [60]:
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(5, 20, num = 4)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf':  [1, 2, 4]}

best_et = hyperparameters_optimization(model = ExtraTreesClassifier(random_state = seed), 
                                       num_folds = 3, param_grid = random_grid, 
                                       X_train = X_train_final, y_train = y_train_final, grid=False)

y_test_pred_et, y_test_proba_et, auc_score_et = train_model_predict(best_et, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_et[:,1])
legend_curve = 'Extra Trees (AUC = ' + str(auc_score_et) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='purple')))

Best: 0.825634 using {'n_estimators': 1600, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}


In [61]:
metrics_et = metrics(y_test = y_test, y_test_pred = y_test_pred_et, auc_score = auc_score_et, model = 'Extra Trees')
metrics_models = pd.concat([metrics_models, metrics_et], axis = 1)

In [62]:
metrics_models

,Logistic Regression,KNeighbors,SVC,Decision Tree,Extra Trees
Precision,0.700,0.64,NaN,0.760,0.720
Recall (TPR),0.250,0.23,0.000,0.200,0.160
FPR,0.020,0.02,0.000,0.010,0.010
AUC,0.808,0.75,0.783,0.768,0.781


In [63]:
feature_importances_model('Extra Trees', best_et, X_train_final, y_train_final)

### Random Forest

In [64]:
random_grid_rf = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)],
                  'max_features': ['auto', 'sqrt'],
                  'max_depth': [int(x) for x in np.linspace(5, 20, num = 4)] + [None],
                  'min_samples_split': [2, 5, 10],
                  'min_samples_leaf':  [1, 2, 4], 
                  'bootstrap': [True, False]}

best_rf = hyperparameters_optimization(model = RandomForestClassifier(random_state = seed), 
                                       num_folds = 3, param_grid = random_grid_rf, 
                                       X_train = X_train_final, y_train = y_train_final, grid=False)

y_test_pred_rf, y_test_proba_rf, auc_score_rf = train_model_predict(best_rf, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_rf[:,1])
legend_curve = 'Random Forest (AUC = ' + str(auc_score_rf) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='magenta')))

Best: 0.835361 using {'n_estimators': 377, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 15, 'bootstrap': True}


In [65]:
metrics_rf = metrics(y_test = y_test, y_test_pred = y_test_pred_rf, auc_score = auc_score_rf, model = 'Random Forest')
metrics_models = pd.concat([metrics_models, metrics_rf], axis = 1)

In [66]:
feature_importances_model('Random Forest', best_rf, X_train_final, y_train_final)

### Gradient Boosting Trees

In [67]:
hyperparameter_grid = {'n_estimators': [100, 500, 900, 1100, 1500],
                       'max_depth': [2, 3, 5, 10, 15],
                       'min_samples_leaf': [1, 2, 4, 6, 8],
                       'min_samples_split': [2, 4, 6, 10],
                       'max_features': ['auto', 'sqrt', 'log2', None]}

best_gbt = hyperparameters_optimization(model = GradientBoostingClassifier(learning_rate=0.1, random_state = seed), 
                                        num_folds = 3, param_grid = hyperparameter_grid, 
                                        X_train = X_train_final, y_train = y_train_final, grid=False)

y_test_pred_gbt, y_test_proba_gbt, auc_score_gbt = train_model_predict(best_gbt, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_gbt[:,1])
legend_curve = 'Gradient Boosting Trees (AUC = ' + str(auc_score_gbt) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='cyan')))

Best: 0.831205 using {'n_estimators': 100, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 5}


In [68]:
metrics_gbt = metrics(y_test = y_test, y_test_pred = y_test_pred_gbt, auc_score = auc_score_gbt, 
                      model = 'Gradient Boosting Trees')
metrics_models = pd.concat([metrics_models, metrics_gbt], axis = 1)

In [69]:
feature_importances_model('Gradient Boosting Trees', best_gbt, X_train_final, y_train_final)

### XGBoost (Extreme Gradient Boosting)

In [70]:
hyperparameter_grid = {'n_estimators': [1000, 1500, 2000],
                       'max_depth': [2, 3, 5, 10, 15],
                       'min_samples_leaf': [1, 2, 4, 6, 8],
                       'min_samples_split': [2, 4, 6, 10],
                       'max_features': ['auto', 'sqrt', 'log2', None], 
                       'learning_rate': [0.05, 0.1, 0.3], 
                       'subsample': [0.8, 0.9, 1.0], 
                       'gamma': [0, 0.25, 0.5, 1]}

best_xgb = hyperparameters_optimization(model = xgb.XGBClassifier(objective = 'binary:logistic', random_state = seed), 
                                        num_folds = 3, param_grid = hyperparameter_grid, 
                                        X_train = X_train_final, y_train = y_train_final, grid=False)

y_test_pred_xgb, y_test_proba_xgb, auc_score_xgb = train_model_predict(best_xgb, X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_test_proba_xgb[:,1])
legend_curve = 'Extreme Gradient Boosting (AUC = ' + str(auc_score_xgb) + ')'
auc_curve.add_trace(go.Scatter(x=fpr, y=tpr, name=legend_curve, mode='lines', line=dict(color='yellow')))

[12:31:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_features, min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:31:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_features, min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:31:25] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_features, min_samples_leaf

[12:36:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:36:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:37:02] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_samples_leaf, min_samples_split } might not be used.



In [71]:
metrics_xgb = metrics(y_test = y_test, y_test_pred = y_test_pred_xgb, auc_score = auc_score_xgb, 
                      model = 'Extreme Gradient Boosting Trees')
metrics_models = pd.concat([metrics_models, metrics_xgb], axis = 1)

In [72]:
feature_importances_model('Extreme Gradient Boosting Trees', best_xgb, X_train_final, y_train_final)

### Models's summary

In [73]:
metrics_models.columns = ['Logistic Regression', 'KNeighbors', 'SVC', 'Decision Tree', 'Extra Trees', 
                          'Random Forest', 'Gradient Boosting Trees', 'Extreme Gradient Boosting Trees']
metrics_models

,Logistic Regression,KNeighbors,SVC,Decision Tree,Extra Trees,Random Forest,Gradient Boosting Trees,Extreme Gradient Boosting Trees
Precision,0.700,0.64,NaN,0.760,0.720,0.790,0.670,0.64
Recall (TPR),0.250,0.23,0.000,0.200,0.160,0.190,0.250,0.27
FPR,0.020,0.02,0.000,0.010,0.010,0.010,0.020,0.02
AUC,0.808,0.75,0.783,0.768,0.781,0.796,0.796,0.79


* **Save models**

In [74]:
joblib.dump(best_logit, '../output_models_ohe/best_logit.pkl')
joblib.dump(best_kNN, '../output_models_ohe/best_knn.pkl')
joblib.dump(best_svm, '../output_models_ohe/best_svm.pkl')
joblib.dump(best_tree, '../output_models_ohe/best_tree.pkl')
joblib.dump(best_et, '../output_models_ohe/best_et.pkl')
joblib.dump(best_rf, '../output_models_ohe/best_rf.pkl')
joblib.dump(best_gbt, '../output_models_ohe/best_gbt.pkl')
joblib.dump(best_xgb, '../output_models_ohe/best_xgb.pkl')

['../output_models_ohe/best_xgb.pkl']

In [75]:
joblib.dump(best_logit, '../output_models/best_logit.pkl')
joblib.dump(best_kNN, '../output_models/best_knn.pkl')
joblib.dump(best_svm, '../output_models/best_svm.pkl')
joblib.dump(best_tree, '../output_models/best_tree.pkl')
joblib.dump(best_et, '../output_models/best_et.pkl')
joblib.dump(best_rf, '../output_models/best_rf.pkl')
joblib.dump(best_gbt, '../output_models/best_gbt.pkl')
joblib.dump(best_xgb, '../output_models/best_xgb.pkl')

['../output_models/best_xgb.pkl']

### Extra

### Voting Classifier

We now apply a stacking model (combining the predictions of the previous models) to see if we can improve the accuracy throught `Soft Voting`. Soft voting entails computing a weighed sum of the predicted probabilities of all models for each class. If the **weights** are equal for all classifiers, then the probabilities are simply the averages for each class.

The predicted label is the argmax of the summed predictino probabilities. Also, **it is common to assign greater weight to the best performing one, but of course, if we assign too much weight to the best classifier then we lose all the benefits of stacking.**

In [76]:
lr = joblib.load('../output_models_ohe/best_logit.pkl')
knn = joblib.load('../output_models_ohe/best_knn.pkl')
svm = joblib.load('../output_models_ohe/best_svm.pkl')
tree = joblib.load('../output_models_ohe/best_tree.pkl')
et = joblib.load('../output_models_ohe/best_et.pkl')
rf = joblib.load('../output_models_ohe/best_rf.pkl')
gbt = joblib.load('../output_models_ohe/best_gbt.pkl')
xgb = joblib.load('../output_models_ohe/best_xgb.pkl')

votclas = VotingClassifier(estimators=[('lr', lr), ('knn', knn), ('svm', svm), ('tree', tree), 
                                       ('et', et), ('rf', rf), ('gbt', gbt), ('xgb', xgb)],
                           voting = 'soft', weights=[4,1,1,1,1,1,2,2])
votclas.fit(X_train_final, y_train_final)

[12:56:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_features, min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




VotingClassifier(estimators=[('lr', LogisticRegression(C=0.1)),
                             ('knn', KNeighborsClassifier(n_neighbors=11)),
                             ('svm',
                              SVC(C=0.0019010851906124285,
                                  gamma=0.00044172017955784455,
                                  probability=True)),
                             ('tree',
                              DecisionTreeClassifier(max_depth=6,
                                                     min_samples_leaf=5,
                                                     min_samples_split=20,
                                                     random_state=123)),
                             ('et',
                              ExtraTreesClassifier(max_depth=20,
                                                   max_features='sqrt...
                                            max_depth=3, max_features='sqrt',
                                            min_child_weight=1,
     

In [77]:
train_df_pred_votclas, y_test_proba_votclas, auc_score_votclas = train_model_predict(votclas, X_test)
print("AUC Voting Classifier:", auc_score_votclas)

AUC Voting Classifier: 0.807


In [78]:
joblib.dump(votclas, '../output_models_ohe/votclf.pkl')

['../output_models_ohe/votclf.pkl']